# Summarizing used Python libraries
This notebook demonstrates how one can dive into summarizing generated code, in this case which Python libraries were used. The `_result.jsonl` files contain a column `completion`, which contains Python code as string. These code snipptes can be summarized for each model as shown here, e.g. by searching for common Python libraries.

In [1]:
import pandas as pd
import os

In [2]:
directory = "../data/"
# if you want to investigate a single model only, add its name here:
search_term = ""

# Enter the terms to search for here
common_libraries = [
'numpy',
'scipy',
'skimage',
'cv2',
'pandas',
'pyclesperanto_prototype',
'vedo',
'umap',
'dask',
'zarr',
'circle_fit',
'statsmodels',
'xarray',
'pillow',
'nibabel',
'SimpleITK',
'trimesh',
'itk']




First we collect all results and the corresponding models from the jsonl files.

In [3]:
collection = []
for filename in os.listdir(directory):
    if search_term in filename and filename.endswith("_results.jsonl"):
        df = pd.read_json(directory + filename, lines=True)
        df['model'] = filename.replace("samples_","").replace("_results","").replace(".jsonl","")
        collection.append(df)

if len(collection) == 1:
    df = collection[0]
else:
    df = pd.concat(collection)

In [4]:
df

,task_id,completion,full_response,result,passed,model
0,../test_cases/apply_otsu_threshold_and_count_p...,\nimport cv2\nimport numpy as np\n\ndef apply_...,Here's the completed code with a step-by-step ...,failed: OpenCV(4.9.0) d:\a\opencv-python\openc...,False,claude-3-opus-20240229
1,../test_cases/binary_closing.ipynb,\nimport numpy as np\nfrom scipy.ndimage impor...,"Here's the completed code with the plan, impor...",passed,True,claude-3-opus-20240229
2,../test_cases/binary_skeleton.ipynb,\nimport numpy as np\nimport cv2\n\ndef binary...,Here's the completed code with a step-by-step ...,failed: module 'cv2' has no attribute 'ximgproc',False,claude-3-opus-20240229
3,../test_cases/bland_altman.ipynb,\n# Plan:\n# 1. Check if the specified columns...,"Here's the completed code with the plan, impor...",passed,True,claude-3-opus-20240229
4,../test_cases/combine_columns_of_tables.ipynb,\n# Plan:\n# 1. Check if the index column is p...,"Here's the completed code with comments, impor...",failed: Can only compare identically-labeled (...,False,claude-3-opus-20240229
...,...,...,...,...,...,...
565,../test_cases/workflow_batch_process_folder_me...,\n import os\n import pandas as pd\n ...,\n import os\n import pandas as pd\n ...,passed,True,reference
566,../test_cases/workflow_segmentation_counting.i...,\n import skimage\n import numpy as np\n...,\n import skimage\n import numpy as np\n...,passed,True,reference
567,../test_cases/workflow_segmentation_measuremen...,\n import skimage\n import numpy as np\n...,\n import skimage\n import numpy as np\n...,passed,True,reference
568,../test_cases/workflow_segment_measure_umap.ipynb,\n import numpy as np\n import pandas as...,\n import numpy as np\n import pandas as...,passed,True,reference


In [5]:
# Just some examples to get an idea
df.head(3)['completion'].tolist()

['\nimport cv2\nimport numpy as np\n\ndef apply_otsu_threshold_and_count_postiive_pixels(image):\n    """\n    Takes an image, applies Otsu\'s threshold method to it to create a binary image and \n    counts the positive pixels.\n    """\n    # Step 1: Convert the image to grayscale\n    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\n    \n    # Step 2: Apply Otsu\'s thresholding method to obtain the threshold value\n    _, threshold_value = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)\n    \n    # Step 3: Create a binary image using the obtained threshold value\n    _, binary_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)\n    \n    # Step 4: Count the number of positive (white) pixels in the binary image\n    positive_pixels = np.sum(binary_image == 255)\n    \n    # Step 5: Return the count of positive pixels\n    return positive_pixels\n',
 '\nimport numpy as np\nfrom scipy.ndimage import binary_dilation, binary_erosion\n

## Searching for common terms
First, we search the generated code for the common libraries as specified above.

In [6]:
# Define the function to count errors
def count_libs(group, lib_list):
    counts = {lib: group['completion'].str.contains(lib, regex=False).sum() for lib in lib_list}
    return pd.Series(counts)

# Apply the function to each model group
lib_counts = df.groupby('model').apply(count_libs, lib_list=common_libraries)

# Transpose the result for the desired format: models as columns, libs as rows
lib_counts = lib_counts.T

C:\Users\haase\AppData\Local\Temp\ipykernel_18760\760873324.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lib_counts = df.groupby('model').apply(count_libs, lib_list=common_libraries)


In [7]:
# reorder columns to be similar to evaluation result plots
with open("model_order.txt", 'r') as f:
    columns_in_order = f.readlines()
    columns_in_order = [c.strip() for c in columns_in_order]
columns_in_order

['reference',
 'gpt-4-turbo-2024-04-09',
 'claude-3-opus-20240229',
 'gpt-4-1106-preview',
 'gpt-3.5-turbo-1106',
 'gemini-pro',
 'codellama']

In [8]:
lib_counts = lib_counts[columns_in_order]
lib_counts

model,reference,gpt-4-turbo-2024-04-09,claude-3-opus-20240229,gpt-4-1106-preview,gpt-3.5-turbo-1106,gemini-pro,codellama
numpy,220,434,453,398,360,165,454
scipy,70,123,131,141,76,31,114
skimage,220,129,125,132,115,116,96
cv2,0,63,44,57,144,82,137
pandas,60,100,99,97,90,52,95
pyclesperanto_prototype,40,0,0,0,0,0,0
vedo,20,0,0,0,0,0,0
umap,20,20,20,20,20,20,20
dask,10,0,0,0,0,0,0
zarr,10,10,10,10,10,10,10


In [9]:
# Iterate over the columns and format new names
new_column_names = [f"\\rot{{{col}}}" for col in lib_counts.columns]

# Update the DataFrame columns
lib_counts.columns = new_column_names
lib_counts

,\rot{reference},\rot{gpt-4-turbo-2024-04-09},\rot{claude-3-opus-20240229},\rot{gpt-4-1106-preview},\rot{gpt-3.5-turbo-1106},\rot{gemini-pro},\rot{codellama}
numpy,220,434,453,398,360,165,454
scipy,70,123,131,141,76,31,114
skimage,220,129,125,132,115,116,96
cv2,0,63,44,57,144,82,137
pandas,60,100,99,97,90,52,95
pyclesperanto_prototype,40,0,0,0,0,0,0
vedo,20,0,0,0,0,0,0
umap,20,20,20,20,20,20,20
dask,10,0,0,0,0,0,0
zarr,10,10,10,10,10,10,10


Next we save the dataframe as tex to disk, so that we can include it in the manuscript.

In [10]:
# Convert DataFrame to TeX format string
tex_str = lib_counts.to_latex()
tex_str = tex_str.replace("_", "\\_")
tex_str = tex_str.replace("prototype", "\\\\ prototype")

# Define filename for the TeX file
tex_filename = '../docs/paper/library_counts_table.tex'

# Write the TeX format string to a file
with open(tex_filename, 'w') as tex_file:
    tex_file.write(tex_str)